In [ ]:
%%writefile Sources/heat-2D.cu
#include "dli.h"

int main()
{
  int height = 4096;
  int width  = 4096;

  thrust::universal_vector<float> prev = dli::init(height, width);
  thrust::universal_vector<float> next(height * width);

  for (int write_step = 0; write_step < 3; write_step++) {
    std::printf("   write step %d\n", write_step);
    dli::store(write_step, height, width, prev);
    
    for (int compute_step = 0; compute_step < 3; compute_step++) {
      auto begin = std::chrono::high_resolution_clock::now();
      dli::simulate(height, width, prev, next);
      auto end = std::chrono::high_resolution_clock::now();
      auto seconds = std::chrono::duration<double>(end - begin).count();
      std::printf("computed step %d in %g s\n", compute_step, seconds);
      prev.swap(next);
    }
  }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/heat-2D.cu # build executable
!/tmp/a.out # run executable

In [ ]:
   write step 0
computed step 0 in 0.0258497 s
computed step 1 in 0.000546186 s
computed step 2 in 0.00053285 s
   write step 1
computed step 0 in 0.0259182 s
computed step 1 in 0.000543439 s
computed step 2 in 0.000533471 s
   write step 2
computed step 0 in 0.0259762 s
computed step 1 in 0.000544832 s
computed step 2 in 0.000532779 s

In [ ]:
%%writefile Sources/heat-2D-explicit-memory-spaces.cu
#include "dli.h"

int main()
{
  int height = 4096;
  int width  = 4096;

  thrust::device_vector<float> d_prev = dli::init(height, width);
  thrust::device_vector<float> d_next(height * width);
  thrust::host_vector<float> h_prev(height * width);

  for (int write_step = 0; write_step < 3; write_step++) {
    std::printf("   write step %d\n", write_step);
    thrust::copy(d_prev.begin(), d_prev.end(), h_prev.begin());
    dli::store(write_step, height, width, h_prev);

    for (int compute_step = 0; compute_step < 3; compute_step++) {
      auto begin = std::chrono::high_resolution_clock::now();
      dli::simulate(height, width, d_prev, d_next);
      auto end = std::chrono::high_resolution_clock::now();
      auto seconds = std::chrono::duration<double>(end - begin).count();
      std::printf("computed step %d in %g s\n", compute_step, seconds);
      d_prev.swap(d_next);
   }
 }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/heat-2D-explicit-memory-spaces.cu # build executable
!/tmp/a.out # run executable

In [ ]:
   write step 0
computed step 0 in 0.000616766 s
computed step 1 in 0.000551491 s
computed step 2 in 0.000551901 s
   write step 1
computed step 0 in 0.000586607 s
computed step 1 in 0.000552583 s
computed step 2 in 0.000550168 s
   write step 2
computed step 0 in 0.000582329 s
computed step 1 in 0.000550868 s
computed step 2 in 0.000550558 s